# CityBikes

Send a request to CityBikes for the city of your choice. 

In [13]:
import requests
import pandas as pd

citybike_networks = requests.get("http://api.citybik.es/v2/networks").json()
#print(citybike_networks)

def get_city_data(city):
    citybike_networks = requests.get("http://api.citybik.es/v2/networks").json()

    list_of_dicts = []
    for citybike_dicts in citybike_networks['networks']:
        new_city = citybike_dicts['location']['city']
        if new_city.lower() == city.lower():
            list_of_dicts.append(citybike_dicts)
    
    return(list_of_dicts)

get_city_data("Vancouver")

[{'id': 'mobibikes',
  'name': 'Mobi',
  'location': {'latitude': 49.2827,
   'longitude': -123.1207,
   'city': 'Vancouver',
   'country': 'CA'},
  'href': '/v2/networks/mobibikes',
  'company': ['Vanncouver Bike Share Inc.',
   'CycleHop LLC',
   'City of Vancouver',
   'Shaw Communications Inc.',
   'Fifteen'],
  'gbfs_href': 'https://vancouver-gbfs.smoove.pro/gbfs/2/gbfs.json'}]

Parse through the response to get the hrefs (networks) to parse. 

In [14]:
def bike_stations_of_city(city):

    stations = get_city_data(city)
    if not stations:
        print("Error: No stations in the city")
        return None
    
    hrefs = stations[0]['href']
    url = "http://api.citybik.es{}".format(hrefs)

    return requests.get(url).json()['network']['stations']

vancouver_stations = bike_stations_of_city("Vancouver")
vancouver_stations[0:2]

[{'id': '00fa94ad698dc4a9e4d708d6fd32f294',
  'name': 'Chilco & Barclay',
  'latitude': 49.291909,
  'longitude': -123.140713,
  'timestamp': '2025-02-21T09:08:21.565238Z',
  'free_bikes': 13,
  'empty_slots': 5,
  'extra': {'uid': '0092',
   'renting': True,
   'returning': True,
   'last_updated': 1740128706,
   'slots': 18,
   'normal_bikes': 9,
   'ebikes': 4,
   'has_ebikes': True}},
 {'id': '012d3e06901cc222b1c2cf0a2ace3a29',
  'name': 'St George & Broadway',
  'latitude': 49.262321,
  'longitude': -123.09306,
  'timestamp': '2025-02-21T09:08:21.604401Z',
  'free_bikes': 1,
  'empty_slots': 13,
  'extra': {'uid': '0248',
   'renting': True,
   'returning': True,
   'last_updated': 1740128022,
   'slots': 14,
   'normal_bikes': 1,
   'ebikes': 0,
   'has_ebikes': True}}]

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes) and put your parsed results into a DataFrame.

In [15]:
def get_stations_details(city):

    stations_details = bike_stations_of_city(city)

    station_list = []
    for details in stations_details:

        a_dict = {
            'latitude': details ['latitude'],
            'longitude': details ['longitude'],
            'free_bikes': details ['free_bikes'],
            'empty_slots': details ['empty_slots'],
            'slots': details ['extra'] ['slots'],
            'uid': details ['extra']['uid'],
        }
    
        station_list.append(a_dict)
    
    return pd.DataFrame(station_list)

vancouver_stations_df = get_stations_details("Vancouver")
vancouver_stations_df.to_pickle("vancouver_stations.pkl")
vancouver_stations_df

,latitude,longitude,free_bikes,empty_slots,slots,uid
0,49.291909,-123.140713,13,5,18,0092
1,49.262321,-123.093060,1,13,14,0248
2,49.275882,-123.071865,0,14,14,0232
3,49.288030,-123.142135,10,16,26,0031
4,49.273777,-123.092723,4,10,14,0233
...,...,...,...,...,...,...
253,49.265089,-123.119425,13,5,18,0081
254,49.277595,-123.095830,10,6,16,0212
255,49.277141,-123.122589,1,11,18,0138
256,49.279821,-123.108020,2,26,28,0053
